<a href="https://colab.research.google.com/github/trafaon/agile-agent/blob/main/agente_nota_fiscal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente de Consulta de Notas Fiscais - LlamaIndex + Groq

Este notebook permite fazer perguntas em linguagem natural sobre dados de notas fiscais (cabeçalho e itens) usando:
- LlamaIndex
- Modelo LLM via Groq
- Embeddings Hugging Face
- Dados CSV de janeiro de 2024

Para uso, configure o arquivo `.env` com sua chave Groq:


In [ ]:
# LlamaIndex com suporte a LLMs externos e leitores de arquivos
!pip install -q llama-index

# Integração com modelo Groq
!pip install -q llama-index-llms-groq

# Embeddings via Hugging Face
!pip install -q llama-index-embeddings-huggingface

# Leitura de arquivos CSV
!pip install -q llama-index-readers-file

# Carregar variáveis de ambiente (.env)
!pip install python-dotenv

In [ ]:
from llama_index.core import Settings
from llama_index.llms import groq

# montar drive
from google.colab import drive
drive.mount('/content/drive')

cabecalho_path = '/content/drive/MyDrive/Colab Notebooks/202401_NFs_Cabecalho.csv'
itens_path = '/content/drive/MyDrive/Colab Notebooks/202401_NFs_Itens.csv'

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
from dotenv import load_dotenv
import os
from pathlib import Path # Import the Path class

# Carregando os CSVs
from llama_index.readers.file import CSVReader

# Importe o modelo de embeddings da Hugging Face
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configurar o modelo de embeddings (usando um modelo comum como exemplo)
# Você pode escolher outro modelo da Hugging Face se preferir
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Configurar o modelo LLM (Groq)
groq_api_key = os.getenv("GROQ_API_KEY")
if groq_api_key is None:
    raise ValueError("GROQ_API_KEY not found in environment variables.")

Settings.llm = Groq(model="llama3-8b-8192", api_key=groq_api_key) # Exemplo de modelo Groq

reader = CSVReader()

# Carregar os documentos de cada arquivo CSV
# Certifique-se de que os paths estão corretos
cabecalho_docs = reader.load_data(file=Path(cabecalho_path))
itens_docs = reader.load_data(file=Path(itens_path))

# Combinar os documentos em uma única lista
docs = cabecalho_docs + itens_docs

# Criando o índice - ele agora usará o embed_model e llm definidos em Settings
index = VectorStoreIndex.from_documents(docs)

# Criando o mecanismo de pergunta
query_engine = index.as_query_engine()

In [ ]:
resposta = query_engine.query("Qual fornecedor recebeu o maior valor?")
print(resposta)